In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
# extra dependencies et-xmlfile-1.1.0 openpyxl-3.1.1

In [2]:
file_path_23 = "./data/23-138.Records.xlsx"
 # skip header rows so column names align, drop all NaN rows
df = pd.read_excel(file_path_23, parse_dates=True, skiprows=6).dropna(how='all')

Sheet1


#### Setting up the Operating Pool DataFrame 

In [3]:
bank_i = df[df['Account or Security'].str.contains("9-200100", na=False)].index
op_i = df[df['Account or Security'].str.contains("Operating Funds Pool", na=False)].index
op_df = df.loc[bank_i[0]:op_i[1]-1]
op_df.insert(6, 'Bank', pd.NA)
op_df.insert(7, 'Asset Type', pd.NA)
op_df.insert(8, 'Company', pd.NA)
op_df.insert(9, 'Industry', pd.NA)
op_df.insert(10, 'Private Placement', False)
# op_df = op_df.insert(7, 'Bank', pd.NA)
op_df.head()

,Account or Security,Coupon,Date,Quantity,Cost Value,Market Value,Bank,Asset Type,Company,Industry,Private Placement
36,9-200100-Neuberger Intermediate,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,False
37,Asset-Backed Securities,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,False
38,CHASE FDG MTG LN 2004-1 MTG LN,2.08,1933-12-25 00:00:00,6462.53,6462.53,6068.88,<NA>,<NA>,<NA>,<NA>,False
39,MVW 2021-2 LLC TIMESHARE LN BK,1.43,1939-05-20 00:00:00,737880.25,737679.54,672033.22,<NA>,<NA>,<NA>,<NA>,False
40,PFS FING CORP .77% 08-15-2026,0.77,2026-08-15 00:00:00,1090000.00,1089872.25,1010524.50,<NA>,<NA>,<NA>,<NA>,False


#### Add the Bank and Asset Type 

In [4]:
bank_name = pd.NA
asset_type = pd.NA
for i in op_df.index:
    if np.isnan(op_df.loc[i]["Quantity"]):
        if "9-200100" in df.loc[i]["Account or Security"]:
            bank_name = df.loc[i]["Account or Security"]
        else:
            asset_type = df.loc[i]["Account or Security"]
    op_df.at[i,'Bank'] = bank_name
    op_df.at[i,'Asset Type'] = asset_type
op_df.head()

,Account or Security,Coupon,Date,Quantity,Cost Value,Market Value,Bank,Asset Type,Company,Industry,Private Placement
36,9-200100-Neuberger Intermediate,NaN,NaN,NaN,NaN,NaN,9-200100-Neuberger Intermediate,<NA>,<NA>,<NA>,False
37,Asset-Backed Securities,NaN,NaN,NaN,NaN,NaN,9-200100-Neuberger Intermediate,Asset-Backed Securities,<NA>,<NA>,False
38,CHASE FDG MTG LN 2004-1 MTG LN,2.08,1933-12-25 00:00:00,6462.53,6462.53,6068.88,9-200100-Neuberger Intermediate,Asset-Backed Securities,<NA>,<NA>,False
39,MVW 2021-2 LLC TIMESHARE LN BK,1.43,1939-05-20 00:00:00,737880.25,737679.54,672033.22,9-200100-Neuberger Intermediate,Asset-Backed Securities,<NA>,<NA>,False
40,PFS FING CORP .77% 08-15-2026,0.77,2026-08-15 00:00:00,1090000.00,1089872.25,1010524.50,9-200100-Neuberger Intermediate,Asset-Backed Securities,<NA>,<NA>,False


In [5]:
all_asset_types = set([op_df.loc[i]['Asset Type'] for i in op_df.index]);all_asset_types

{<NA>,
 'Account Totals',
 'Account Totals - Allspring Conservative Income Fund',
 'Account Totals - Allspring Short Duration',
 'Account Totals - Galliard Capital',
 'Account Totals - Income Research Intermediate',
 'Account Totals - Income Research Short Duration',
 'Account Totals - JP Morgan Managed Reserves ',
 'Account Totals - Neuberger Berman Enhanced Cash',
 'Account Totals - Neuberger Intermediate',
 'Account Totals - Northern Trust UI Short Duration',
 'Asset Backed Securities',
 'Asset-Backed Securities',
 'Cash & Cash Equivalents',
 'Commercial Mortgage Backed Securities',
 'Commercial Paper',
 'Corporate Bonds',
 'International Government Bonds',
 'Municipal Bonds',
 'U.S. Agency Bonds',
 'U.S. Treasury Securities'}

#### First, we just check the corperate bonds 

In [6]:
cb_df = op_df[op_df['Asset Type'].str.contains("Corporate Bonds", na=False)]
print("Corperate Bond Totals")
print("Cost Value\t",'${:,.2f}'.format(cb_df.sum(numeric_only=True)["Cost Value"]))
print("Market Value\t",'${:,.2f}'.format(cb_df.sum(numeric_only=True)["Market Value"]))

Corperate Bond Totals
Cost Value	 $1,020,500,769.44
Market Value	 $985,000,673.71


Yup, you read that right

##### get company name
+ Parse our 'PVTPL', which is an abreviation for privatly placed https://www.investopedia.com/terms/p/privateplacement.asp
+ Remove everything after and including the tokens ```['%']```
+ Remove everything after `[" CAP", " INC", " FDG", " CORP", " CO", " LLC", " CR"]`
+ Add the company names to a set
+ Map different semantic names to the same syntax for the same company

In [7]:
company_name_dict = {
    "AMERICAN EXPRESS" : "AMERICAN EXPRESS CO",
    'AIG GLOBAL' : "AMERICAN INTL GROUP  INC",
    "ANHEUSER-BUSCH" : "ANHEUSER-BUSCH CO",
    "APTIV" : "APTIV CO",
    "ASTRAZENECA" : "ASTRAZENECA PLC",
    "AUSTRALIA & NEW" : "AUSTRALIA & NEW ZEALAND BKG GR",
    "BAE SYS" : "BAE SYS PLC",
    "BANCO SANTANDER" : "BANCO SANTANDER SA",
    "BANK MONTREAL" : "BANK OF MONTREAL",
    "BANK OF MONTREAL" : "BANK OF MONTREAL",
    "BK MONTREAL" : "BANK OF MONTREAL",
    "BANK NOVA SCOTIA" : "BANK OF NOVA SCOTIA",
    "BANK OF NOVA SCOTIA" : "BANK OF NOVA SCOTIA",
    "BANK AMER" : "BANK OF AMERICA CO",
    "BAXTER INT" : "BAXTER INTERNATIONAL INC",
    "BAYER US FIN" : "BAYER US FINANCE LLC",
    "BB&T" : "BB&T CO",
    "BLACKSTONE" : "BLACKSTONE",
    "BMW" : "BMW",
    "BNP PARIBAS" : "BNP PARIBAS",
    "BRIGHTHOUSE" : "BRIGHTHOUSE",
    "BRISTOL MYERS SQUIBB" : "BRISTOL MYERS SQUIBB CO",
    "BRISTOL-MYERS SQUIBB" : "BRISTOL MYERS SQUIBB CO",
    "CANADIAN IMPERIAL BK" : "CANADIAN IMPERIAL BK CO",
    "CAPITAL ONE" : "CAPITAL ONE FINL CO",
    "CATERPILLAR FINL" : "CATERPILLAR FINL",
    "CENTERPOINT ENERGY" : "CENTERPOINT ENERGY INC",
    "CHEVRON U S A" : "CHEVRON  CO",
    "CREDIT AGRICOLE" : "CREDIT AGRICOLE",
    "CREDIT SUISSE" : "CREDIT SUISSE GROUP AG",
    "CROWN CASTLE INTL" : "CROWN CASTLE INTL",
    "DAIMLER" : "DAIMLER",
    "DELTA AIR LINES" : "DELTA AIR LINES",
    "DTE E" : "DTE ELEC",
    "DUKE ENERGY" : "DUKE ENERGY CO",
    "DOWDUPONT  INC" : "DUPONT DE NEMOURS  INC",
    "ENTERGY" : "ENTERGY CO",
    "EQUITABLE FINL LIFE" : "EQUITABLE FINL LIFE GLOBAL FDG",
    "ESC CB LEHMAN BROS" : "ESC LEHMAN BROTH HLD  INC",
    "FIFTH THIRD BANCORP" : "FIFTH THIRD BANCORP",
    "FLORIDA P" : "FLORIDA POWER & LIGHT CO",
    "GENERAL MTRS" : "GENERAL MOTORS",
    "GENERAL MOTORS" : "GENERAL MOTORS",
    "HEWLETT PACKARD" : "HEWLETT PACKARD ENTERPRISE CO",
    "HP  INC" : "HEWLETT PACKARD ENTERPRISE CO",
    "HUNTINGTON" : "HUNTINGTON NATL BK MD",
    "JACKSON FINANCIAL  INC" : "JACKSON NATIONAL LIFE GL",
    "JPM CHASE" : "JPMORGAN CHASE &  CO",
    "KINDER MORGAN" : "KINDER MORGAN INC",
    "LLOYDS BKG" : "LLOYDS BANKING GROUP PLC FORME",
    "MACQUARIE" : "MACQUARIE BK LTD",
    "MIZUHO" : "MIZUHO CO",
    "MONDELEZ INT" : "MONDELEZ INTERNATIONAL INC",
    "MORGAN STANLEY" : "MORGAN STANLEY",
    "NATIONAL AUSTRALIA B" : "NATIONAL AUSTRALIA BANK",
    "NATIONWIDE BLDG SOC" : "NATIONWIDE BLDG SOCIETY",
    "NATIONAL BANK OF CANADA" : "NATIONAL BANK OF CANADA",
    "NATL BK CDA" : "NATIONAL BANK OF CANADA",
    "NATWEST M" : "NATWEST MARKETS PLC",
    "NEXTERA ENERGY" : "NEXTERA ENERGY CAP",
    "NORDEA BANK" : "NORDEA BANK",
    "NORTHWESTERN" : "NORTHWESTERN MUT",
    "NXP B V" : "NXP B V",
    "PHILLIPS 66" : "PHILLIPS 66",
    "PRINCIPAL LIFE GLOBAL" : "PRINCIPAL LIFE GLOBAL FDG",
    "PROTECTIVE LIFE G" : "PROTECTIVE LIFE GLOBAL",
    "PUBLIC SVC" : "PUBLIC SERVICE ELECTRIC & GAS",
    "RABOBANK NEDERLAND" : "RABOBANK NEDERLAND",
    "ROCHE H" : "ROCHE HOLDINGS INC",
    "ROPER " : "ROPER TECHNOLOGIES  INC",
    "ROYAL BANK OF CANADA" : "ROYAL BANK OF CANADA",
    "ROYAL BK CDA" : "ROYAL BANK OF CANADA",
    "SCHLUMBERGER" : "SCHLUMBERGER", 
    "SIEMENS FINANCIERINGSMAA" : "SIEMENS FINANCIERINGSMAATSCHAP",
    "SIMON PPTY GROUP" : "SIMON PPTY GROUP",
    "STATE STR" : "STATE STREET CO",
    "SUMITOMO MITSUI" : "SUMITOMO MITSUI BANKING",
    "SWEDBANK AB" : "SWEDBANK AB",
    "TORONTO DOMINION B" : "TORONTO DOMINION BANK",
    "TOYOTA M" : "TOYOTA MOTOR",
    "TRANS-CDA PIPELINES" : "TRANS-CDA PIPELINES",
    "TRUIST " : "TRUIST BANK",
    "TSMC " : "TSMC GLOBAL LTD",
    "U S B" : "U S BANCORP",
    "UBS " : "UBS AG LONDON",
    "VENTAS REALTY" : "VENTAS REALTY LP",
    "VOLKSWAGEN GROUP" : "VOLKSWAGEN GROUP",
    "WESTPAC B" : "WESTPAC BANKING",
}

In [8]:
company_names = set()
for i in cb_df.index:
    # Set the Company to be cleaned
    cb_df.at[i,'Company'] = cb_df.at[i,'Account or Security']
    if not np.isnan(cb_df.at[i,'Quantity']):
        # clean private placement
        for prefix in ["PVTPL", "PVPTL", "PVYPL", "PVT PL", "PVPTL"]:
            if prefix in cb_df.loc[i]["Company"]:
                cb_df.at[i,'Private Placement'] = True
                cb_df.at[i,'Company'] = cb_df.at[i,'Company'][6:].strip()
        for end in [" CAP", " INC", " FDG", " CORP", " CO", " LLC", " CR", " SR", " A/S", " LP", " ASA", " LTD", ]:
            if end in cb_df.at[i, "Company"]:
                cb_df.at[i, "Company"] = cb_df.at[i, 'Company'].split(end)[0].strip()+" "+end
        for token in ['%']:
            if token in cb_df.at[i, "Company"]:
                # get everythng before the token, then get eveything before the last space
                cb_df.at[i, "Company"] = cb_df.at[i, "Company"].split(token)[0].rsplit(' ', 1)[0].strip()
        for key, value in company_name_dict.items():
            if key in cb_df.at[i, "Company"]:
                cb_df.at[i, "Company"] = value
        company_names.add(cb_df.at[i, "Company"])
    else:
        cb_df.drop(i, axis=0)


In [9]:
len(company_names)

408

It looks like we are invested in 408 companies

In [10]:
cb_df

,Account or Security,Coupon,Date,Quantity,Cost Value,Market Value,Bank,Asset Type,Company,Industry,Private Placement
103,Corporate Bonds,NaN,NaN,NaN,NaN,NaN,9-200100-Neuberger Intermediate,Corporate Bonds,Corporate Bonds,<NA>,False
104,ABBVIE INC 2.6% DUE 11-21-2024,2.6,2024-11-21 00:00:00,1065000.0,1135158.13,1030877.86,9-200100-Neuberger Intermediate,Corporate Bonds,ABBVIE INC,<NA>,False
105,ABBVIE INC 2.95% DUE,2.95,2026-11-21 00:00:00,495000.0,545836.80,469035.55,9-200100-Neuberger Intermediate,Corporate Bonds,ABBVIE INC,<NA>,False
106,AERCAP IRELAND CAP / GLOBA 3.0,3,2028-10-29 00:00:00,440000.0,439199.20,370620.29,9-200100-Neuberger Intermediate,Corporate Bonds,AERCAP IRELAND CAP,<NA>,False
107,AERCAP IRELAND CAP / GLOBA,3.56,1932-01-30 00:00:00,430000.0,428383.20,344211.28,9-200100-Neuberger Intermediate,Corporate Bonds,AERCAP IRELAND CAP,<NA>,False
...,...,...,...,...,...,...,...,...,...,...,...
2706,UBS AG LONDON BRH NT FLTG RATE,1.39,2024-08-09 00:00:00,1766000.0,1766000.00,1754003.40,9-200100-JP Morgan Managed Reserves,Corporate Bonds,UBS AG LONDON,<NA>,False
2707,UBS GROUP FDG SWITZ AG NT FLTG,2.36,2023-08-15 00:00:00,2000000.0,1999460.00,1999374.24,9-200100-JP Morgan Managed Reserves,Corporate Bonds,UBS AG LONDON,<NA>,False
2708,UNITED PARCEL SVC INC 2.45% DU,2.45,2022-10-01 00:00:00,145000.0,148575.70,144922.71,9-200100-JP Morgan Managed Reserves,Corporate Bonds,UNITED PARCEL SVC INC,<NA>,False
2709,WESTPAC BKG CORP 2.0% DUE 01-1,2,2023-01-13 00:00:00,72000.0,73785.60,71637.73,9-200100-JP Morgan Managed Reserves,Corporate Bonds,WESTPAC BANKING,<NA>,False


##### get ticker

In [11]:
# Replace "company_name" with the name of the company you're searching for
company_name = "ABBV"

# Search for the company on Yahoo Finance
search_results = yf.Tickers(company_name)

# Get the ticker symbol for the first result
print (search_results.tickers['ABBV'].info['industry'])

# print(ticker)

Drug Manufacturers—General
